In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import os
import sys
from importlib import reload
from scipy.stats import zscore
import scipy.io
from scipy.interpolate import make_interp_spline, BSpline
from scipy.signal import savgol_filter as savgol 
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import leidenalg
import umap
import gget
from sklearn import preprocessing

from pydmd import DMD

import matplotlib.pyplot as plt
import seaborn as sns

# local imports
sys.path.append("../python/")

import utils as ut
reload(ut)

import hasnain_DMD
reload(hasnain_DMD)

2023-10-09 15:44:12.970776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 15:44:13.085218: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-09 15:44:13.085242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-09 15:44:13.112573: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 15:44:15.341985: W tensorflow/stream_executor/pla

<module 'hasnain_DMD' from '/home/cstansbu/git_repositories/DMD_gene/notebooks/../python/hasnain_DMD.py'>

In [7]:
raw_path = f"/nfs/turbo/umms-indikar/shared/projects/time_series_rna/chen2015/RNA_pipeline_ouputs/chen2015_rna/countMatrix/counts.raw.txt"


""" Load the raw expression """
df = pd.read_csv(raw_path, index_col=0)
print(f"{df.shape=}")
df.head()

df.shape=(19393, 18)


,S1a,S1b,S2a,S2b,S3a,S3b,S4a,S4b,S5a,S5b,S6a,S6b,S7a,S7b,S8a,S8b,S9a,S9b
geneName,,,,,,,,,,,,,,,,,,
A1BG,12,5,5,9,2,6,7,3,6,5,5,5,4,3,4,5,5,3
A1CF,0,0,0,0,0,0,0,0,0,0,0,1,2,0,2,0,0,0
A2M,0,0,0,2,0,0,0,0,0,0,1,0,1,0,0,0,0,2
A2ML1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# build fold-change dataframe

def fc(df):
    """A function to compute fold-changes over initial time points """
    x0 = df[df.columns[0]]

    for c in df.columns:
        df[c] = (df[c] + 1) /  (x0 + 1)

    return df

a = [x for x in df.columns if "a" in x]
b = [x for x in df.columns if "b" in x]

dfa = df[a]
dfb = df[b]

print(f"{dfa.shape=} {dfa.shape=}")

dfa = fc(dfa)
dfa.head()

dfa.shape=(19393, 9) dfa.shape=(19393, 9)


/tmp/ipykernel_975071/2333121175.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c] = (df[c] + 1) /  (x0 + 1)


,S1a,S2a,S3a,S4a,S5a,S6a,S7a,S8a,S9a
geneName,,,,,,,,,
A1BG,1.0,0.461538,0.230769,0.615385,0.538462,0.461538,0.384615,0.384615,0.461538
A1CF,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,1.000000
A2M,1.0,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000
A2ML1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
A3GALT2,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# exf = exf.reset_index(drop=False)
# exf['replicate'] = exf['index'].apply(lambda x: x.split("_")[1])
# exf['time'] = exf['index'].apply(lambda x: x.split("_")[0].replace('T', ''))